### Recommending a game to a user

In [8]:
import mysql.connector 

connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "password",
    database = "steam_data"
)


In [9]:
sql = "SELECT COUNT(*) FROM steam_purchase;"
print(sql)

SELECT COUNT(*) FROM steam_play;


In [10]:
cursor = connection.cursor()
cursor.execute(sql)
result = cursor.fetchall()
print(result)
cursor.close()

[(70488,)]


In [11]:
from graphtastic.database.mysql import query_mysql

The steam_purchase table contains information on user purchases. 

In [14]:
result = query_mysql("SELECT * FROM steam_purchase LIMIT 5;", password=PASSWORD)
print(result)

[('151603712', 'Fallout 4', 'purchase', 1.0, 0), ('151603712', 'Spore', 'purchase', 1.0, 0), ('151603712', 'Fallout New Vegas', 'purchase', 1.0, 0), ('151603712', 'Left 4 Dead 2', 'purchase', 1.0, 0), ('151603712', 'HuniePop', 'purchase', 1.0, 0), ('151603712', 'Path of Exile', 'purchase', 1.0, 0), ('151603712', 'Poly Bridge', 'purchase', 1.0, 0), ('151603712', 'Left 4 Dead', 'purchase', 1.0, 0), ('151603712', 'Team Fortress 2', 'purchase', 1.0, 0), ('151603712', 'Tomb Raider', 'purchase', 1.0, 0)]


In [15]:
for row in result:
    print(row)

('151603712', 'Fallout 4', 'purchase', 1.0, 0)
('151603712', 'Spore', 'purchase', 1.0, 0)
('151603712', 'Fallout New Vegas', 'purchase', 1.0, 0)
('151603712', 'Left 4 Dead 2', 'purchase', 1.0, 0)
('151603712', 'HuniePop', 'purchase', 1.0, 0)
('151603712', 'Path of Exile', 'purchase', 1.0, 0)
('151603712', 'Poly Bridge', 'purchase', 1.0, 0)
('151603712', 'Left 4 Dead', 'purchase', 1.0, 0)
('151603712', 'Team Fortress 2', 'purchase', 1.0, 0)
('151603712', 'Tomb Raider', 'purchase', 1.0, 0)


In the steam_price table, the first column again contains a unique user ID. As with steam_play, the second column contains a game's name. The third column always contains the purchase string. The fourth column in this table always contains floatt 1.0, indicating the purchase of that row's game, by the user.

### Path-based analytics in tabular data

Now that we have an idea of what our tables contain, we can ask some questions about our data. For
example, we can ask, “What other games may a user enjoy, based on their previous purchase habits?”
This type of question forms the basis of a recommendation system, used across the sales and marketing
industry to drive additional customer purchases.

At a glance, we can see that there is data for user 151603712 in both the steam_play
and steam_purchase tables. So, users in our datasets have data about both the purchase
of games and play time. From a brief look into the data, we can see that there are plenty of
columns that we won’t need going forward, so we can start to exclude them from our analysis
by querying specific rows.